# Nettoyage de "***la Grosse Bertha***"

### Import modules

In [2]:
import sqlite3

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

### Connexion à "***la Grosse Bertha_1.1***" en **LOCAL** sur mon poste

In [3]:
path = 'D:/tmp storage/Project#3/SQLite3 db/GrosseBertha_1.1.db'
db_conn = sqlite3.connect(path)

## Exploration des tables

### Filtrage des films
On cherche à ne garder dans la database **que les films présents initialement dans la base de données de notre client**.
On supprimera donc toutes les entrées de la base IMDB qui ne sont pas dans la base du client.

#### Table **links**


In [3]:
query = '''
SELECT * FROM links
LIMIT 1
'''
pd.read_sql(query, db_conn)

,movieId,imdbId,tmdbId
0,172063,1,16612


La table **links** comporte les **id** des films utilisés dans la base client et dans la base IMDB.
___
Première ligne de la table **movies** correnspondant au film "*Toy Story*"


In [4]:
query = '''
SELECT * FROM movies
LIMIT 1
'''
pd.read_sql(query, db_conn)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


La ligne de la table **imdb_title_basics** correspondante au film "*Toy Story*" (l'équivalent de la table movies pour la base imdb)

In [5]:
query = '''
SELECT * FROM imdb_title_basics
WHERE primaryTitle == 'Toy Story' AND titleType == 'movie'
'''
pd.read_sql(query, db_conn)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0114709,movie,Toy Story,Toy Story,0,1995,\N,81,"Adventure,Animation,Comedy"


On constate que **`tconst`**, qui représente l'id du film dans cette table, n'a pas tout à fait le format attendu..

|**id dans links** | **id dans imdb** |
|------------------|------------------|
| 114709           | tt0114709         |

Il faut donc convertir les **id** de la table **links** pour qu'ils correspondent à ceux utilisés dans la base IMDB
___
On créer un DF à partir de **links**

In [6]:
query = '''
SELECT * FROM links
'''
links = pd.DataFrame(pd.read_sql(query, db_conn))

In [7]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   imdbId   58098 non-null  int64 
 2   tmdbId   58098 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [8]:
links.shape

(58098, 3)

On ajoute des **`00`** pour avoir 7 chiffres et **`tt`** à chaque **`imdbId`**

In [9]:
links['imdbId'] = links['imdbId'].apply(lambda x: str(x).zfill(7))
links['imdbId'] = links['imdbId'].apply(lambda x: 'tt' + x)

In [10]:
links.head()

,movieId,imdbId,tmdbId
0,172063,tt0000001,16612
1,140539,tt0000003,88013
2,95541,tt0000005,16624
3,180695,tt0000007,159895
4,88674,tt0000008,105158


In [11]:
links.tail()

,movieId,imdbId,tmdbId
58093,192795,tt8816130,527266
58094,193273,tt8837018,539033
58095,193277,tt8846862,537973
58096,193279,tt8916694,485164
58097,193849,tt9038520,549443


Et on remplace la table **links** par le DataFrame ***links***

In [12]:
links.to_sql('links', db_conn, if_exists = 'replace', index = False)

In [13]:
query = '''
SELECT * FROM links
'''
pd.DataFrame(pd.read_sql(query, db_conn)).shape

(58098, 3)

On constate que la table link comporte maintenant le double de lignes. Il faut donc supprimer


## Filtrage des films
On souhaite ne garder que les films qui sont dans la base de données de notre client pour ne pas proposer des films qu'il ne pocède pas.

### SELECT
On sélectionne tous les **films de la base IMDB** dont l'identifiant se trouve également dans la table **links**

In [14]:
query = '''
SELECT * FROM imdb_title_basics
WHERE tconst IN
    (SELECT imdbId
     FROM links)
'''
imdb_mouvies = pd.DataFrame(pd.read_sql(query, db_conn))
imdb_mouvies.shape

(58016, 9)

Il y a donc 58016 films en commun (il semblerait que certains films n'ait pas leur id dans la table IMBD)
___
### INSERT dans la BDD
On remplace la table **`imdb_title_basics`** par le DF qui ne contient que les films client.

In [15]:
imdb_mouvies.to_sql('imdb_title_basics', db_conn, if_exists = 'replace', index = False)

In [16]:
query = '''
SELECT * FROM imdb_title_basics
'''
pd.DataFrame(pd.read_sql(query, db_conn)).shape

(58016, 9)

## Nettoyage des "ratings"

### SELECT
On ne sélectionne que les ratings portant sur les films qui sont dans notre BDD

In [17]:
query = '''
SELECT * FROM imdb_title_ratings Ir
WHERE Ir.tconst IN (
    SELECT imdbId FROM links
    )
'''
imdb_ratings = pd.DataFrame(pd.read_sql(query, db_conn))
imdb_ratings.shape

(57928, 3)

### Import to BDD

In [18]:
imdb_ratings.to_sql('imdb_title_ratings', db_conn, if_exists = 'replace', index = False)

In [19]:
query = '''
SELECT * FROM imdb_title_ratings
'''
pd.DataFrame(pd.read_sql(query, db_conn)).shape

(57928, 3)

## Nettoyage des "title_principal"

### SELECT
On ne sélectionne que les données portant sur les films qui sont dans notre BDD

In [3]:
query = '''
SELECT * FROM imdb_title_principals titles
WHERE titles.tconst IN (
    SELECT imdbId FROM links
    );
'''
imdb_titles = pd.DataFrame(pd.read_sql(query, db_conn))
imdb_titles.shape

(550362, 6)

In [7]:
imdb_titles.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000003,1,nm0721526,director,\N,\N
4,tt0000003,2,nm5442194,producer,producer,\N


### Import to BDD

In [4]:
imdb_titles.to_sql('imdb_title_principals', db_conn, if_exists = 'replace', index = False)

## Base des acteurs

### SELECT

In [5]:
query = '''
SELECT * FROM imdb_name_basic names
WHERE names.nconst IN (
    SELECT p.nconst
        FROM imdb_title_principals p
    );
'''
imdb_names = pd.DataFrame(pd.read_sql(query, db_conn))
imdb_names.shape

(210057, 6)

### Import dans la BDD

In [6]:
imdb_names.to_sql('imdb_name_basic', db_conn, if_exists = 'replace', index = False)
